In [1]:
!pip install dash

In [2]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

C:\Users\Bin Lyu\AppData\Local\Temp\ipykernel_11068\1439708741.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\Bin Lyu\AppData\Local\Temp\ipykernel_11068\1439708741.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [3]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [4]:
# Create a dash application
app = dash.Dash(__name__)


In [7]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(
                                    id='site-dropdown',
                                    options=[
                                        {'label':'All Sites','value':'ALL'},
                                        {'label':'CCAFS LC-40','value':'LC40'},
                                        {'label':'VAFB SLC-4E','value':'SLC4E'},
                                        {'label':'KSC LC-39A','value':'LC39A'},
                                        {'label':'CCAFS SLC-40','value':'SLC40'}
                                    ],
                                    value='ALL',
                                    placeholder='Select a Launch Site here',
                                    searchable=True
                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(
                                    id='payload-slider',
                                    min=0,
                                    max=10000,
                                    step=1000,
                                    marks = {
                                        0:'0',
                                        2500:'2500',
                                        5000:'5000',
                                        7500:'7500',
                                        10000:'10000'
                                    },
                                    value=[min_payload,max_payload]
                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:   
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(
        component_id='success-pie-chart',
        component_property='figure'
    ),
    Input(
        component_id='site-dropdown',
        component_property='value'
    )
)
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        filtered_df = spacex_df
        fig = px.pie(filtered_df,values='class',\
            names='Launch Site',title='Total Success Launches By Site')
    elif entered_site == 'LC40':
        filtered_df = spacex_df[spacex_df['Launch Site'] == 'CCAFS LC-40']
        fig = px.pie(filtered_df,names='class',\
            title='Total Success Launches for site CCAFS LC-40')
    elif entered_site == 'SLC4E':
        filtered_df = spacex_df[spacex_df['Launch Site'] == 'VAFB SLC-4E']
        fig = px.pie(filtered_df,names='class',\
            title='Total Success Launches for site VAFB SLC-4E')
    elif entered_site == 'LC39A':
        filtered_df = spacex_df[spacex_df['Launch Site'] == 'KSC LC-39A']
        fig = px.pie(filtered_df,names='class',\
            title='Total Success Launches for site KSC LC-39A')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == 'CCAFS SLC-40']
        fig = px.pie(filtered_df,names='class',\
            title='Total Success Launches for site CCAFS SLC-40')
    return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(
        component_id='success-payload-scatter-chart',
        component_property='figure'
    ),
    [
        Input(
            component_id='site-dropdown',
            component_property='value'
        ),
        Input(
            component_id='payload-slider',
            component_property='value'
        )
    ]
)
def get_scatter_chart(entered_site,payload_mass):
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload_mass[0]) &\
        (spacex_df['Payload Mass (kg)'] <= payload_mass[1])]
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df,x='Payload Mass (kg)',y='class',\
            color='Booster Version Category',\
                title='Correlation between Payload and Success for all Sites')
    elif entered_site == 'LC40':
        filtered_df = filtered_df[filtered_df['Launch Site'] == 'CCAFS LC-40']
        fig = px.scatter(filtered_df,x='Payload Mass (kg)',y='class',\
            color='Booster Version Category',\
                title='Correlation between Payload and Success for site CCAFS LC-40')
    elif entered_site == 'SLC4E':
        filtered_df = filtered_df[filtered_df['Launch Site'] == 'VAFB SLC-4E']
        fig = px.scatter(filtered_df,x='Payload Mass (kg)',y='class',\
            color='Booster Version Category',\
                title='Correlation between Payload and Success for site VAFB SLC-4E')
    elif entered_site == 'LC39A':
        filtered_df = filtered_df[filtered_df['Launch Site'] == 'KSC LC-39A']
        fig = px.scatter(filtered_df,x='Payload Mass (kg)',y='class',\
            color='Booster Version Category',\
                title='Correlation between Payload and Success for site KSC LC-39A')
    else:
        filtered_df = filtered_df[filtered_df['Launch Site'] == 'CCAFS SLC-40']
        fig = px.scatter(filtered_df,x='Payload Mass (kg)',y='class',\
            color='Booster Version Category',\
                title='Correlation between Payload and Success for site CCAFS SLC-40')
    return fig

In [8]:
app.run_server()